In [3]:
import sys
from pathlib import Path

currentWorkingDirectory = Path('')
projectRootDirectory = currentWorkingDirectory.parent
sys.path.append(projectRootDirectory.as_posix())

In [7]:
import pandas as pd
import numpy as np
import numexpr as ne

from IPython.display import Markdown, display

In [8]:
dataPath = Path("../data/raw/")
interimDataPath = Path("../data/interim/")

def loadData(duplicateN:int=1):
    df = pd.read_csv(dataPath / "atlanticInterpolated.csv")
    z = df.z.values
    dropColumns = []
    for col in df.columns:
        if df.loc[30, col] > df.loc[0, col] - 0.1:
            dropColumns.append(col)
    df = df.drop(columns=dropColumns)
    df = pd.concat([df for _ in range(duplicateN)], axis=1)
    temps = df.iloc[:, 1:].values
    surfaceTemps = temps[:2, :].mean(axis=0)
    return df, temps, z, surfaceTemps

In [14]:
df, temps, z, surfaceTemps = loadData(duplicateN=5)

In [15]:
thresholdTemperatureDifference = 0.1

In [16]:
%timeit -n 1 -r 1 (temps < (surfaceTemps - thresholdTemperatureDifference)).nonzero()

82.3 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [17]:
%timeit -n 1 -r 1 ne.evaluate("temps < (surfaceTemps - thresholdTemperatureDifference)").nonzero()

27.6 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
